In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False)
data_text=data[['headline_text']]
data_text['index']=data_text.index
documents=data_text

In [3]:
documents.head()

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


### Data Precessing

In [4]:
import nltk
from nltk.tokenize import sent_tokenize

In [5]:
### Sentence tokenization
text='Statistics skills, and programming skills are equally important for analytics. Statistics skills, and domain knowledge are important for analytics. I like reading books and travelling.'

sent_tokenize_list = sent_tokenize(text)
print(sent_tokenize_list)

['Statistics skills, and programming skills are equally important for analytics.', 'Statistics skills, and domain knowledge are important for analytics.', 'I like reading books and travelling.']


In [7]:
# Another equivalent call method using TreebankWordTokenizer
from nltk.tokenize import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()
print(tokenizer.tokenize(text))

['Statistics', 'skills', ',', 'and', 'programming', 'skills', 'are', 'equally', 'important', 'for', 'analytics.', 'Statistics', 'skills', ',', 'and', 'domain', 'knowledge', 'are', 'important', 'for', 'analytics.', 'I', 'like', 'reading', 'books', 'and', 'travelling', '.']


In [8]:
## remove numbers
import re
def remove_numbers(text):
    return re.sub(r'\d+', '', text)
text = 'This is a    sample  English  sentence, \n with whitespace and numbers 1234!'
print('Removed numbers: ', remove_numbers(text))

Removed numbers:  This is a    sample  English  sentence, 
 with whitespace and numbers !


In [9]:
## remove punctuations
import string
# Function to remove punctuations
def remove_punctuations(text):
    words = nltk.word_tokenize(text)
    punt_removed = [w for w in words if w.lower() not in string.punctuation]
    return " ".join(punt_removed)

remove_punctuations('This is a sample English sentence, with punctuations!')

'This is a sample English sentence with punctuations'

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/ling/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [13]:
# Function to remove stop words
def remove_stopwords(text, lang='english'):
    words = nltk.word_tokenize(text)
    lang_stopwords = stopwords.words(lang)
    stopwords_removed = [w for w in words if w.lower() not in lang_stopwords]
    return " ".join(stopwords_removed)

remove_stopwords('This is a sample English sentence')

'sample English sentence'

In [14]:
# Function to remove whitespace
def remove_whitespace(text):
    return " ".join(text.split())
text = 'This is a     sample  English   sentence, \n with whitespace and numbers 1234!'
print('Removed whitespace: ', remove_whitespace(text))

Removed whitespace:  This is a sample English sentence, with whitespace and numbers 1234!


In [21]:
from nltk import chunk
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ling/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/ling/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/ling/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [22]:
tagged_sent = nltk.pos_tag(nltk.word_tokenize('This is a sample English sentence'))
print(tagged_sent)

tree = chunk.ne_chunk(tagged_sent)
tree.draw() # this will draw the sentence tree

[('This', 'DT'), ('is', 'VBZ'), ('a', 'DT'), ('sample', 'JJ'), ('English', 'JJ'), ('sentence', 'NN')]


In [24]:
from nltk.tag.perceptron import PerceptronTagger
PT = PerceptronTagger()
print(PT.tag('This is a sample English sentence'.split()))

[('This', 'DT'), ('is', 'VBZ'), ('a', 'DT'), ('sample', 'JJ'), ('English', 'JJ'), ('sentence', 'NN')]


### STEMMING

In [26]:
from nltk import PorterStemmer, LancasterStemmer, SnowballStemmer

In [39]:
# Function to apply stemming to a list of words
def words_stemmer(words, type="PorterStemmer", lang="english", encoding="utf8"):
    supported_stemmers = ["PorterStemmer","LancasterStemmer","SnowballStemmer"]
    if type is False or type not in supported_stemmers:
        return words
    else:
        stem_words = []
        if type == "PorterStemmer":
            stemmer = PorterStemmer()
            for word in words:
                stem_words.append(stemmer.stem(word).encode(encoding))
        if type == "LancasterStemmer":
            stemmer = LancasterStemmer()
            for word in words:
                stem_words.append(stemmer.stem(word).encode(encoding))
        if type == "SnowballStemmer":
            stemmer = SnowballStemmer(lang)
            for word in words:
                stem_words.append(stemmer.stem(word).encode(encoding))
        return "".join(str(stem_words))
    

In [44]:
words =  'caring cares cared caringly carefully'

print("Original: " + words)
print("Porter: " + words_stemmer(nltk.word_tokenize(words)) + "PorterStemmer")
print("Lancaster: " + words_stemmer(nltk.word_tokenize(words)) + "LancasterStemmer")
print("Snowball: " +words_stemmer(nltk.word_tokenize(words)) + "SnowballStemmer")


Original: caring cares cared caringly carefully
Porter: [b'care', b'care', b'care', b'caringli', b'care']PorterStemmer
Lancaster: [b'care', b'care', b'care', b'caringli', b'care']LancasterStemmer
Snowball: [b'care', b'care', b'care', b'caringli', b'care']SnowballStemmer


### LEMMATIZATION

In [46]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [50]:
# Function to apply lemmatization to a list of words
def words_lemmatizer(text, encoding="utf8"):
    words = nltk.word_tokenize(text)
    lemma_words = []
    wl = WordNetLemmatizer()
    for word in words:
        pos = find_pos(word)
        lemma_words.append(wl.lemmatize(word, pos).encode(encoding))
    return "".join(str(lemma_words))

In [51]:
# Function to find part of speech tag for a word
def find_pos(word):
    # Part of Speech constants
    # ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'
    # You can learn more about these at http://wordnet.princeton.edu/wordnet/man/wndb.5WN.html#sect3
    # You can learn more about all the penn tree tags at https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
    pos = nltk.pos_tag(nltk.word_tokenize(word))[0][1]
    # Adjective tags - 'JJ', 'JJR', 'JJS'
    if pos.lower()[0] == 'j':
        return 'a'
    # Adverb tags - 'RB', 'RBR', 'RBS'
    elif pos.lower()[0] == 'r':
        return 'r'
    # Verb tags - 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'
    elif pos.lower()[0] == 'v':
        return 'v'
    # Noun tags - 'NN', 'NNS', 'NNP', 'NNPS'
    else:
        return 'n'

In [52]:
print("Lemmatized: " + words_lemmatizer(words))

Lemmatized: [b'care', b'care', b'care', b'caringly', b'carefully']


### N-GRAMS

In [53]:
from nltk.util import ngrams
from collections import Counter

In [71]:
# Function to extract n-grams from text
def get_ngrams(text, n):
    n_grams = ngrams(nltk.word_tokenize(text), n)
    return [.join(grams) for grams in n_grams]

SyntaxError: invalid syntax (<ipython-input-71-355179e01617>, line 4)

In [72]:
ls = get_ngrams(text, 1)

In [69]:
[]

In [70]:
res

[('This',), ('is',), ('a',), ('sample',), ('English',), ('sentence',)]

In [ ]:
return ''.join(str(grams)) for grams in n_grams]

In [60]:
text = 'This is a sample English sentence'
print("1-gram: " + get_ngrams(text, 1))
print("2-gram: " + get_ngrams(text, 2))
print("3-gram: " + get_ngrams(text, 3))
print("4-gram: " + get_ngrams(text, 4))

TypeError: can only concatenate str (not "list") to str

In [12]:
import numpy as np
np.random.seed(2018)

In [27]:
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /Users/ling/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/ling/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [40]:
# def lemmatize_stemming(text):
#     lemmatizer = nltk.WordNetLemmatizer()
# #     return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
#     processed_tokens = nltk.word_tokenize(text)
#     processed_tokens = [w.lower() for w in processed_tokens]
#     for w in processed_tokens:
#         processed_tokens.append(lemmatizer.lemmatize(w)) 
#     return processed_tokens

def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

In [45]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [42]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfires']


In [43]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0              [decide, community, broadcast, licence]
1                         [witness, aware, defamation]
2           [call, infrastructure, protection, summit]
3                          [staff, aust, strike, rise]
4             [strike, affect, australian, travellers]
5               [ambitious, olsson, win, triple, jump]
6               [antic, delight, record, break, barca]
7    [aussie, qualifier, stosur, waste, memphis, ma...
8             [aust, address, security, council, iraq]
9                         [australia, lock, timetable]
Name: headline_text, dtype: object

In [44]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [45]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [46]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(112, 1), (489, 1), (970, 1), (4427, 1)]

In [47]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], dictionary[bow_doc_4310[i][0]], bow_doc_4310[i][1]))

Word 112 ("help") appears 1 time.
Word 489 ("rain") appears 1 time.
Word 970 ("bushfires") appears 1 time.
Word 4427 ("dampen") appears 1 time.


## LDA using Bag of Words

In [48]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.595557507312157),
 (1, 0.3948343404468723),
 (2, 0.4976240764382213),
 (3, 0.4917187993528624)]


In [49]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [50]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.032*"charge" + 0.030*"court" + 0.030*"sydney" + 0.022*"murder" + 0.020*"face" + 0.016*"jail" + 0.016*"years" + 0.015*"shoot" + 0.015*"accuse" + 0.015*"drug"
Topic: 1 
Words: 0.020*"market" + 0.014*"share" + 0.012*"fall" + 0.011*"news" + 0.010*"port" + 0.010*"build" + 0.009*"industry" + 0.009*"marriage" + 0.009*"energy" + 0.009*"say"
Topic: 2 
Words: 0.036*"trump" + 0.031*"government" + 0.016*"donald" + 0.013*"hour" + 0.013*"deal" + 0.012*"return" + 0.011*"abuse" + 0.010*"say" + 0.009*"young" + 0.009*"china"
Topic: 3 
Words: 0.021*"home" + 0.015*"tasmanian" + 0.014*"high" + 0.014*"price" + 0.013*"women" + 0.012*"flood" + 0.012*"bank" + 0.012*"guilty" + 0.011*"rise" + 0.011*"royal"
Topic: 4 
Words: 0.031*"queensland" + 0.023*"coast" + 0.020*"live" + 0.015*"claim" + 0.015*"gold" + 0.014*"north" + 0.012*"force" + 0.011*"green" + 0.011*"fear" + 0.011*"party"
Topic: 5 
Words: 0.055*"police" + 0.022*"kill" + 0.021*"crash" + 0.018*"die" + 0.017*"interview" + 0.016*"miss" + 0

In [52]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.4199962615966797	 
Topic: 0.015*"plan" + 0.014*"health" + 0.014*"fund" + 0.014*"council" + 0.012*"help" + 0.012*"water" + 0.011*"say" + 0.009*"service" + 0.009*"park" + 0.009*"budget"

Score: 0.22000005841255188	 
Topic: 0.044*"australia" + 0.023*"election" + 0.021*"south" + 0.014*"test" + 0.013*"power" + 0.013*"tasmania" + 0.012*"people" + 0.012*"turnbull" + 0.009*"island" + 0.009*"violence"

Score: 0.21999981999397278	 
Topic: 0.032*"charge" + 0.030*"court" + 0.030*"sydney" + 0.022*"murder" + 0.020*"face" + 0.016*"jail" + 0.016*"years" + 0.015*"shoot" + 0.015*"accuse" + 0.015*"drug"

Score: 0.020002376288175583	 
Topic: 0.036*"trump" + 0.031*"government" + 0.016*"donald" + 0.013*"hour" + 0.013*"deal" + 0.012*"return" + 0.011*"abuse" + 0.010*"say" + 0.009*"young" + 0.009*"china"

Score: 0.020001482218503952	 
Topic: 0.021*"home" + 0.015*"tasmanian" + 0.014*"high" + 0.014*"price" + 0.013*"women" + 0.012*"flood" + 0.012*"bank" + 0.012*"guilty" + 0.011*"rise" + 0.011*"royal"

S

In [54]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.35000014305114746	 Topic: 0.036*"trump" + 0.031*"government" + 0.016*"donald" + 0.013*"hour" + 0.013*"deal"
Score: 0.1833333820104599	 Topic: 0.017*"state" + 0.017*"country" + 0.016*"rural" + 0.014*"hospital" + 0.012*"change"
Score: 0.18333332240581512	 Topic: 0.032*"charge" + 0.030*"court" + 0.030*"sydney" + 0.022*"murder" + 0.020*"face"
Score: 0.18333317339420319	 Topic: 0.044*"australia" + 0.023*"election" + 0.021*"south" + 0.014*"test" + 0.013*"power"
Score: 0.01666666753590107	 Topic: 0.020*"market" + 0.014*"share" + 0.012*"fall" + 0.011*"news" + 0.010*"port"
Score: 0.01666666753590107	 Topic: 0.021*"home" + 0.015*"tasmanian" + 0.014*"high" + 0.014*"price" + 0.013*"women"
Score: 0.01666666753590107	 Topic: 0.031*"queensland" + 0.023*"coast" + 0.020*"live" + 0.015*"claim" + 0.015*"gold"
Score: 0.01666666753590107	 Topic: 0.055*"police" + 0.022*"kill" + 0.021*"crash" + 0.018*"die" + 0.017*"interview"
Score: 0.01666666753590107	 Topic: 0.020*"canberra" + 0.020*"open" + 0.015

## LDA using TF-IDF

In [51]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.019*"rural" + 0.014*"crash" + 0.008*"royal" + 0.007*"fatal" + 0.006*"die" + 0.006*"truck" + 0.005*"commission" + 0.005*"road" + 0.004*"police" + 0.004*"rail"
Topic: 1 Word: 0.023*"country" + 0.020*"hour" + 0.007*"christmas" + 0.007*"mother" + 0.005*"live" + 0.005*"history" + 0.005*"politics" + 0.005*"wrap" + 0.005*"bail" + 0.005*"prison"
Topic: 2 Word: 0.008*"marriage" + 0.007*"friday" + 0.007*"wednesday" + 0.006*"asylum" + 0.006*"august" + 0.006*"great" + 0.005*"capital" + 0.005*"wild" + 0.004*"children" + 0.004*"allegedly"
Topic: 3 Word: 0.010*"news" + 0.008*"health" + 0.007*"budget" + 0.007*"business" + 0.006*"fund" + 0.006*"say" + 0.005*"sport" + 0.005*"national" + 0.005*"victorian" + 0.005*"monday"
Topic: 4 Word: 0.007*"john" + 0.007*"october" + 0.007*"september" + 0.005*"outback" + 0.004*"scott" + 0.004*"footage" + 0.004*"refugees" + 0.004*"speed" + 0.003*"young" + 0.003*"knight"
Topic: 5 Word: 0.009*"australia" + 0.007*"world" + 0.007*"grandstand" + 0.006*"abbot

In [53]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.2947407364845276	 
Topic: 0.012*"coast" + 0.009*"turnbull" + 0.009*"weather" + 0.007*"queensland" + 0.007*"search" + 0.007*"flood" + 0.006*"north" + 0.006*"miss" + 0.006*"south" + 0.006*"november"

Score: 0.289143830537796	 
Topic: 0.008*"marriage" + 0.007*"friday" + 0.007*"wednesday" + 0.006*"asylum" + 0.006*"august" + 0.006*"great" + 0.005*"capital" + 0.005*"wild" + 0.004*"children" + 0.004*"allegedly"

Score: 0.27610644698143005	 
Topic: 0.019*"rural" + 0.014*"crash" + 0.008*"royal" + 0.007*"fatal" + 0.006*"die" + 0.006*"truck" + 0.005*"commission" + 0.005*"road" + 0.004*"police" + 0.004*"rail"

Score: 0.020002324134111404	 
Topic: 0.010*"news" + 0.008*"health" + 0.007*"budget" + 0.007*"business" + 0.006*"fund" + 0.006*"say" + 0.005*"sport" + 0.005*"national" + 0.005*"victorian" + 0.005*"monday"

Score: 0.020001716911792755	 
Topic: 0.007*"john" + 0.007*"october" + 0.007*"september" + 0.005*"outback" + 0.004*"scott" + 0.004*"footage" + 0.004*"refugees" + 0.004*"speed" + 0.

In [55]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.359101802110672	 Topic: 0.055*"police" + 0.022*"kill" + 0.021*"crash" + 0.018*"die" + 0.017*"interview"
Score: 0.3326616883277893	 Topic: 0.020*"canberra" + 0.020*"open" + 0.015*"australian" + 0.013*"world" + 0.013*"league"
Score: 0.19155389070510864	 Topic: 0.032*"charge" + 0.030*"court" + 0.030*"sydney" + 0.022*"murder" + 0.020*"face"
Score: 0.01667175441980362	 Topic: 0.021*"home" + 0.015*"tasmanian" + 0.014*"high" + 0.014*"price" + 0.013*"women"
Score: 0.016671014949679375	 Topic: 0.044*"australia" + 0.023*"election" + 0.021*"south" + 0.014*"test" + 0.013*"power"
Score: 0.016668202355504036	 Topic: 0.036*"trump" + 0.031*"government" + 0.016*"donald" + 0.013*"hour" + 0.013*"deal"
Score: 0.016668176278471947	 Topic: 0.031*"queensland" + 0.023*"coast" + 0.020*"live" + 0.015*"claim" + 0.015*"gold"
Score: 0.016667908057570457	 Topic: 0.017*"state" + 0.017*"country" + 0.016*"rural" + 0.014*"hospital" + 0.012*"change"
Score: 0.01666787639260292	 Topic: 0.020*"market" + 0.014*"sha